##### Note: Running the whole notebook takes around 5 minutes due to looping the API calls.

## Setup

In [1]:
# import dependencies
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import json
from pprint import pprint
from api_keys import gkey
import re
from pandas import *
import sqlalchemy
from sqlalchemy import create_engine

# load in csv files
county_crime_csv = "input/2010 CA County Crime Data.csv"
county_officer_csv = "input/2010 CA County LEO Data.csv"
city_crime_csv = "input/2010 CA City Crime Data.csv"
city_officer_csv = "input/2010 CA City LEO Data.csv"
population_data_csv = "input/CA Census Population Data.csv"
county_area_csv = "input/County Land Area.csv"

# read the csv files to data frames
county_crime_df = pd.read_csv(county_crime_csv)
county_officers_df = pd.read_csv(county_officer_csv)
city_crime_df = pd.read_csv(city_crime_csv)
city_officers_df = pd.read_csv(city_officer_csv)
population_df = pd.read_csv(population_data_csv)
counties_df = pd.read_csv(county_area_csv)

## Clean the County Crime Data

In [2]:
# drop extraneuos rows before the header row
county_crime_df = county_crime_df.drop([0,1,2,3]).reset_index(drop=True)

# define the headers, remove their row, and insert the headers as column names
headers = county_crime_df.iloc[0]
county_crime_df = county_crime_df[1:]
county_crime_df.columns = headers
county_crime_df = county_crime_df.reset_index(drop=True)

# remove '\n' from column names and rename the 'county type' column header
county_crime_df.columns = county_crime_df.columns.str.strip().str.replace('\n',' ').str.replace('  ',' ')
county_crime_df.rename(columns={'Metropolitan/Nonmetropolitan':'County Type'}, inplace=True)

# fill the county type down to replace the NaN values and remove 'Counties' from the end of the county type
county_crime_df['County Type'].fillna(method='ffill', inplace=True)
county_crime_df['County Type'] = county_crime_df['County Type'].str.split().str.get(0)

# remove Tulare County (because of incomplete data) and the last row with NaN values from a footnote
county_crime_df = county_crime_df.dropna().reset_index(drop=True)

# moving the county name in front of the county type
county_crime_df.rename(columns={'County':'County old'}, inplace=True)
county_crime_df.insert(0, 'County', county_crime_df['County old'])

# remove the old 'County' column
del county_crime_df['County old']

# removing Arson column as it is not included in property crime totals
del county_crime_df['Arson']

# store list of columns that need to be converted to numeric format
col_list = list(county_crime_df.iloc[:,2:11].columns)

# loop through the columns, remove commas, and change to integers
for item in col_list:
    county_crime_df[item] = county_crime_df[item].str.replace(',', '').astype(int)

In [3]:
# display the county crime data frame
county_crime_df.head()

,County,County Type,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft
0,Alameda,Metropolitan,698,8,18,235,437,2212,622,1041,549
1,Butte,Metropolitan,133,4,18,24,87,1296,564,721,11
2,Contra Costa,Metropolitan,449,12,28,119,290,2754,1011,1729,14
3,El Dorado,Metropolitan,256,3,15,34,204,1990,864,1114,12
4,Fresno,Metropolitan,727,10,33,116,568,4442,1466,2230,746


## Clean the City Crime Data & Pull Corresponding Counties from Google Maps API

In [4]:
# drop extraneuos rows before the header row
city_crime_df = city_crime_df.drop([0,1,2]).reset_index(drop=True)

# define the headers, remove their row, and insert the headers as column names
headers = city_crime_df.iloc[0]
city_crime_df = city_crime_df[1:]
city_crime_df.columns = headers
city_crime_df = city_crime_df.reset_index(drop=True)

# clean column headers and delete extraneous columns
city_crime_df.columns = city_crime_df.columns.str.strip().str.replace('\n',' ')
del city_crime_df['Population']
del city_crime_df['Arson']

# drop extra rows that have NaN values
city_crime_df = city_crime_df.dropna().reset_index(drop=True)

# insert column to store county names
city_crime_df.insert(1, 'County', '')

# base url for google maps api calls
base_url = 'https://maps.googleapis.com/maps/api/geocode/json?key='

# loop through data frame, call the api for the city, and pull out the county into the 'County' column
for index, row in city_crime_df.iterrows():
    
    city = row['City']
    
    query_url = base_url + gkey + '&address=' + city + ',California'
    
    response = requests.get(query_url).json()
    
    if((response['results'][0]['address_components'][1]['long_name'].rsplit(' ', 1)[-1]) == 'County'):
        county = response['results'][0]['address_components'][1]['long_name']
    else:
        county = response['results'][0]['address_components'][2]['long_name']
   
    city_crime_df.loc[index, 'County'] = county.rsplit(' ', 1)[0]

# store list of columns that need to be converted to numeric format
col_list = list(city_crime_df.iloc[:,2:11].columns)

# loop through the columns, remove commas, and change to integers
for item in col_list:
    city_crime_df[item] = city_crime_df[item].str.replace(',', '').astype(int)

# group the cities by county and sum the crime counts
grouped_city_crime_df = city_crime_df.groupby(['County']).sum()

# remove Tulare County from the data frame
grouped_city_crime_df = grouped_city_crime_df.drop('Tulare').reset_index()

In [5]:
# display the new data frame of city agency crimes grouped by county
grouped_city_crime_df.head()

,County,Violent crime,Murder and nonnegligent manslaughter,Forcible rape,Robbery,Aggravated assault,Property crime,Burglary,Larceny- theft,Motor vehicle theft
0,Alameda,9610,124,501,4532,4453,45335,11086,25781,8468
1,Alpine,1,0,1,0,0,72,9,60,3
2,Amador,50,0,3,4,43,326,87,220,19
3,Butte,618,3,71,129,415,4168,1262,2544,362
4,Colusa,26,0,2,3,21,217,59,135,23


## Clean the County Police Officer Data

In [6]:
# drop extraneuos rows before the header row
county_officers_df = county_officers_df.drop([0,1,2]).reset_index(drop=True)

# define the headers, remove their row, and insert the headers as column names
headers = county_officers_df.iloc[0]
county_officers_df = county_officers_df[1:]
county_officers_df.columns = headers
county_officers_df = county_officers_df.reset_index(drop=True)

# clean the column headers and delete extraneous columns
county_officers_df.columns = county_officers_df.columns.str.strip().str.replace('\n',' ')
del county_officers_df['Metropolitan/Nonmetropolitan']
del county_officers_df['Total law enforcement employees']
del county_officers_df['Total civilians']

# remove Tulare County
county_officers_df = county_officers_df[county_officers_df['County'] != 'Tulare']
county_officers_df = county_officers_df.dropna().reset_index(drop=True)

# change the data type of the officer count to integer
county_officers_df['Total officers'] = county_officers_df['Total officers'].str.replace(',', '').astype(int)

In [7]:
# display the county officers data frame
county_officers_df.head()

,County,Total officers
0,Alameda,986
1,Butte,101
2,Contra Costa,637
3,El Dorado,173
4,Fresno,693


## Clean the City Police Officer Data & Pull Corresponding Counties from Google Maps API

In [8]:
# drop extraneuos rows before the header row
city_officers_df = city_officers_df.drop([0,1,2]).reset_index(drop=True)

# define the headers, remove their row, and insert the headers as column names
headers = city_officers_df.iloc[0]
city_officers_df = city_officers_df[1:]
city_officers_df.columns = headers
city_officers_df = city_officers_df.reset_index(drop=True)

# clean column headers and delete extraneous columns
city_officers_df.columns = city_officers_df.columns.str.strip().str.replace('\n',' ')
del city_officers_df['Population']
del city_officers_df['Total law enforcement employees']
del city_officers_df['Total civilians']

# drop extra rows that have NaN values
city_officers_df = city_officers_df.dropna().reset_index(drop=True)

city_officers_df.insert(1, 'County', '')

# base url for google maps api calls
base_url = 'https://maps.googleapis.com/maps/api/geocode/json?key='

# loop through data frame, call the api for the city, and pull out the county into the 'County' column

for index, row in city_officers_df.iterrows():
    
    city = row['City']
    
    query_url = base_url + gkey + '&address=' + city + ',California'
    
    response = requests.get(query_url).json()
    
    if((response['results'][0]['address_components'][1]['long_name'].rsplit(' ', 1)[-1]) == 'County'):
        county = response['results'][0]['address_components'][1]['long_name']
    else:
        county = response['results'][0]['address_components'][2]['long_name']
    
    city_officers_df.loc[index, 'County'] = county.rsplit(' ', 1)[0]
    
# change data type of the officer count to integer
city_officers_df['Total officers'] = city_officers_df['Total officers'].str.replace(',', '').astype(int)

# group the cities by county and sum the officer counts
grouped_city_officers_df = city_officers_df.groupby(['County']).sum().reset_index()

# remove Tulare County 
grouped_city_officers_df = grouped_city_officers_df[grouped_city_officers_df['County'] != 'Tulare']

In [9]:
# display the new data frame of city officers grouped by county
grouped_city_officers_df.head()

,County,Total officers
0,Alameda,1765
1,Alpine,8
2,Amador,20
3,Butte,159
4,Colusa,19


## Merge Crime Data Frames

In [10]:
# create new data frame that will be used to merge crime data
total_crime_df = county_crime_df

# merge county crime with the grouped city crime
total_crime_df = total_crime_df.merge(grouped_city_crime_df, on='County', how='outer', suffixes=('_county', '_grouped')).fillna(0)

# add Metropolitan label to San Francisco and Solano, which weren't listed in the county crime data
total_crime_df['County Type'] = total_crime_df['County Type'].replace(0, 'Metropolitan')

# loop through columns and sum ones with the same first word and insert into new column
for item in county_crime_df.iloc[:,2:11].columns:
    total = total_crime_df.filter(regex=re.split('\s|_|-',item)[0]).sum(axis=1).astype(int)
    total_crime_df[item.title()] = total
    
# delete old columns from the merge
for item in total_crime_df.iloc[:,2:20].columns:
    del total_crime_df[item]

# replace double spaces with single spaces
total_crime_df.columns = total_crime_df.columns.str.replace('  ',' ')

In [11]:
# display the total crime data frame
total_crime_df.head()

,County,County Type,Violent Crime,Murder And Nonnegligent Manslaughter,Forcible Rape,Robbery,Aggravated Assault,Property Crime,Burglary,Larceny- Theft,Motor Vehicle Theft
0,Alameda,Metropolitan,10308,132,519,4767,4890,47547,11708,26822,9017
1,Butte,Metropolitan,751,7,89,153,502,5464,1826,3265,373
2,Contra Costa,Metropolitan,4257,65,187,1525,2480,28491,7546,15494,5451
3,El Dorado,Metropolitan,485,3,34,66,382,2614,1112,1460,42
4,Fresno,Metropolitan,4904,63,181,1358,3302,38442,8982,22680,6780


## Merge Police Officer Data Frames

In [12]:
# create new dataframe for officer data to be merged into
total_officers_df = county_officers_df

# merge county officer data with the grouped city officer data
total_officers_df = total_officers_df.merge(grouped_city_officers_df, on='County', how='outer', suffixes=('_county', '_grouped'))

# replace NaN values with zero and add officer columns to get total
total_officers_df.fillna(0, inplace=True)
total_officers_df['Total Officers'] = (total_officers_df['Total officers_county'] + total_officers_df['Total officers_grouped']).astype(int)

# delete columns we no longer need
del total_officers_df['Total officers_county'] 
del total_officers_df['Total officers_grouped']

In [13]:
# display the total officers data frame
total_officers_df.head()

,County,Total Officers
0,Alameda,2751
1,Butte,260
2,Contra Costa,1612
3,El Dorado,230
4,Fresno,1793


## Merge Police Officer Data into Crime Data Frame

In [14]:
# merge total crime with total officers
crime_officer_data = total_crime_df.merge(total_officers_df, on='County')

In [15]:
# display the crime and officer data frame
crime_officer_data.head()

,County,County Type,Violent Crime,Murder And Nonnegligent Manslaughter,Forcible Rape,Robbery,Aggravated Assault,Property Crime,Burglary,Larceny- Theft,Motor Vehicle Theft,Total Officers
0,Alameda,Metropolitan,10308,132,519,4767,4890,47547,11708,26822,9017,2751
1,Butte,Metropolitan,751,7,89,153,502,5464,1826,3265,373,260
2,Contra Costa,Metropolitan,4257,65,187,1525,2480,28491,7546,15494,5451,1612
3,El Dorado,Metropolitan,485,3,34,66,382,2614,1112,1460,42,230
4,Fresno,Metropolitan,4904,63,181,1358,3302,38442,8982,22680,6780,1793


## Clean County Population Data & Merge into the Crime & Officer Data Frame

In [16]:
# drop extraneuos rows before the header row
population_df = population_df.drop([0,2,3]).reset_index(drop=True)

# define the headers, remove their row, and insert the headers as column names
headers = population_df.iloc[0]
population_df = population_df[1:]
population_df.columns = headers
population_df = population_df.reset_index(drop=True)

# remove '\n' from column headers
population_df.columns = population_df.columns.str.strip().str.replace('\n','')

# extract County Name, Population for 2010, and sumlvl
population_df = population_df[['County','April 1,2010','sumlvl']]

# remove all rows that are not at the county level
population_df = population_df.loc[population_df['sumlvl'] == '050'].reset_index(drop=True)

# remove the sumlvl column and rename the date of the population to 'Population'
del population_df['sumlvl']
population_df = population_df.rename(columns={'April 1,2010':'Population'})

# remove Tulare County
population_df = population_df.loc[population_df['County'] != 'Tulare']
# remove Klamath County, NaN because it no longer exists
population_df = population_df.dropna().reset_index(drop=True)

# change the data type of 'Population' into an integer
population_df['Population'] = population_df['Population'].str.replace(',', '').astype(int)

# merge the population column into the crime and officer data frame
crime_officer_pop = crime_officer_data.merge(population_df, on='County')

In [17]:
# display the crime, officer, and population data frame
crime_officer_pop.head()

,County,County Type,Violent Crime,Murder And Nonnegligent Manslaughter,Forcible Rape,Robbery,Aggravated Assault,Property Crime,Burglary,Larceny- Theft,Motor Vehicle Theft,Total Officers,Population
0,Alameda,Metropolitan,10308,132,519,4767,4890,47547,11708,26822,9017,2751,1510271
1,Butte,Metropolitan,751,7,89,153,502,5464,1826,3265,373,260,220000
2,Contra Costa,Metropolitan,4257,65,187,1525,2480,28491,7546,15494,5451,1612,1049025
3,El Dorado,Metropolitan,485,3,34,66,382,2614,1112,1460,42,230,181058
4,Fresno,Metropolitan,4904,63,181,1358,3302,38442,8982,22680,6780,1793,930450


## Clean County Land Area Data & Merge into the Crime, Officer, & Population Data Frame

In [18]:
# limit county land area data down to California counties
ca_counties_df = counties_df.loc[(counties_df['STCOU'] > 6000) & (counties_df['STCOU'] < 6200)]
ca_counties_df = ca_counties_df.reset_index(drop=True)

# extract the county name and land area for 2010
ca_counties_df = ca_counties_df[['Areaname','LND110210D']]

# define the county from the areaname column
county = ca_counties_df['Areaname'].str.split(',',1, expand=True)

# insert the county into a new column
ca_counties_df['County'] = county[0]

# insert land area into a new column and format as an integer
ca_counties_df['Land Area'] = ca_counties_df['LND110210D'].astype(int)

# delete old columns
del ca_counties_df['Areaname']
del ca_counties_df['LND110210D']

# remove Tulare County
ca_counties_df = ca_counties_df.loc[ca_counties_df['County'] != 'Tulare'].reset_index(drop=True)

In [19]:
# display the California counties land area data frame
ca_counties_df.head()

,County,Land Area
0,Alameda,739
1,Alpine,738
2,Amador,594
3,Butte,1636
4,Calaveras,1020


## Prep Data Frames for Output to SQLite

In [20]:
# insert columns to store latitude and longitude for each city
city_crime_df.insert(1, 'Lat', '')
city_crime_df.insert(2, 'Lng', '')

# base url for google maps api calls
base_url = 'https://maps.googleapis.com/maps/api/geocode/json?key='

# loop through data frame, call the api for the city, and pull out the county into the 'County' column
for index, row in city_crime_df.iterrows():
    
    city = row['City']
    
    query_url = base_url + gkey + '&address=' + city + ',California'
    
    response = requests.get(query_url).json()
    
    lat = response['results'][0]['geometry']['location']['lat']
    lng = response['results'][0]['geometry']['location']['lng']
    
    city_crime_df.loc[index, 'Lat'] = lat
    city_crime_df.loc[index, 'Lng'] = lng

# change Lat and Lng to float to retain all decimal places
city_crime_df.loc[:,'Lat'] = city_crime_df.loc[:,'Lat'].astype(float)
city_crime_df.loc[:,'Lng'] = city_crime_df.loc[:,'Lng'].astype(float)

# merge in officer data and remove extra county column
city_output = city_crime_df.merge(city_officers_df, on='City')
city_output.rename(columns={'County_x':'County'}, inplace=True)
del city_output['County_y']

# loop through columns and format the names to match the total output data frame
for item in city_output.columns:
    city_output.rename(columns={item: item.title()}, inplace=True)

# format column headers for SQLite
city_output.columns = city_output.columns.str.replace('-', '').str.replace(' ', '_')

# insert and calculate a column for total crime
city_output.insert(4, 'Total_Crime', '')
city_output['Total_Crime'] = city_output['Violent_Crime'] + city_output['Property_Crime']

# remove Tulare County
city_output = city_output[city_output['County'] != 'Tulare']

In [21]:
city_output.head()

,City,Lat,Lng,County,Total_Crime,Violent_Crime,Murder_And_Nonnegligent_Manslaughter,Forcible_Rape,Robbery,Aggravated_Assault,Property_Crime,Burglary,Larceny_Theft,Motor_Vehicle_Theft,Total_Officers
0,Alameda,37.779872,-122.282185,Alameda,2137,173,1,15,74,83,1964,389,1360,215,85
1,Albany,37.886704,-122.297756,Alameda,619,37,0,5,24,8,582,95,392,95,25
2,Alhambra,34.095287,-118.127015,Los Angeles,2260,185,3,8,82,92,2075,345,1460,270,84
3,Alturas,41.487115,-120.542456,Modoc,68,5,0,2,0,3,63,28,30,5,7
4,Anaheim,33.836593,-117.914301,Orange,9634,1161,7,88,492,574,8473,1594,5869,1010,389


In [29]:
# merge land area into crime, officer, population dataframe to make the county data frame
county_output = crime_officer_pop.merge(ca_counties_df, on='County')

# sort by county type and county to move the two metropolitan counties from the bottom 
county_output = county_output.sort_values(by=['County Type','County']).reset_index(drop=True)

# insert columns to store latitude and longitude for each county
county_output.insert(2, 'Lat', '')
county_output.insert(3, 'Lng', '')

# call Google Maps API to get coordinates for each county
for index, row in county_output.iterrows():
    
    county = row['County'] + ' County'
    
    query_url = base_url + gkey + '&address=' + county + ',California'
    
    response = requests.get(query_url).json()
    
    lat = response['results'][0]['geometry']['location']['lat']
    lng = response['results'][0]['geometry']['location']['lng']
    
    county_output.loc[index, 'Lat'] = lat
    county_output.loc[index, 'Lng'] = lng

# change Lat and Lng to float to retain all decimal places
county_output.loc[:,'Lat'] = county_output.loc[:,'Lat'].astype(float)
county_output.loc[:,'Lng'] = county_output.loc[:,'Lng'].astype(float)

# format column headers for SQLite
county_output.columns = county_output.columns.str.replace('-', '').str.replace(' ', '_')

# insert and calculate a column for total crime
county_output.insert(4, 'Total_Crime', '')
county_output['Total_Crime'] = county_output['Violent_Crime'] + county_output['Property_Crime']

In [35]:
# display the county data frame
county_output.head()

,County,County_Type,Lat,Lng,Total_Crime,Violent_Crime,Murder_And_Nonnegligent_Manslaughter,Forcible_Rape,Robbery,Aggravated_Assault,Property_Crime,Burglary,Larceny_Theft,Motor_Vehicle_Theft,Total_Officers,Population,Land_Area
0,Alameda,Metropolitan,37.601689,-121.719546,57855,10308,132,519,4767,4890,47547,11708,26822,9017,2751,1510271,739
1,Butte,Metropolitan,39.625395,-121.537000,6215,751,7,89,153,502,5464,1826,3265,373,260,220000,1636
2,Contra Costa,Metropolitan,37.853409,-121.901795,32748,4257,65,187,1525,2480,28491,7546,15494,5451,1612,1049025,715
3,El Dorado,Metropolitan,38.742638,-120.435763,3099,485,3,34,66,382,2614,1112,1460,42,230,181058,1707
4,Fresno,Metropolitan,36.985898,-119.232078,43346,4904,63,181,1358,3302,38442,8982,22680,6780,1793,930450,5957


In [44]:
# create a data frame to be used to calculate crime rates
county_rates = county_output.copy()

# calculate the crime rate for each type of crime
for x in range(4,14):
    county_rates.iloc[:,x] = (county_rates.iloc[:,x]/county_rates['Population'] * 10000).round(3)

In [45]:
county_rates.head()

,County,County_Type,Lat,Lng,Total_Crime,Violent_Crime,Murder_And_Nonnegligent_Manslaughter,Forcible_Rape,Robbery,Aggravated_Assault,Property_Crime,Burglary,Larceny_Theft,Motor_Vehicle_Theft,Total_Officers,Population,Land_Area
0,Alameda,Metropolitan,37.601689,-121.719546,383.077,68.253,0.874,3.436,31.564,32.378,314.824,77.523,177.597,59.705,2751,1510271,739
1,Butte,Metropolitan,39.625395,-121.537000,282.500,34.136,0.318,4.045,6.955,22.818,248.364,83.000,148.409,16.955,260,220000,1636
2,Contra Costa,Metropolitan,37.853409,-121.901795,312.176,40.581,0.620,1.783,14.537,23.641,271.595,71.933,147.699,51.963,1612,1049025,715
3,El Dorado,Metropolitan,38.742638,-120.435763,171.161,26.787,0.166,1.878,3.645,21.098,144.374,61.417,80.637,2.320,230,181058,1707
4,Fresno,Metropolitan,36.985898,-119.232078,465.861,52.706,0.677,1.945,14.595,35.488,413.155,96.534,243.753,72.868,1793,930450,5957


## Push DataFrames to SQLite Database

In [46]:
# create connection to sqlite database
conn = create_engine('sqlite:///db/CA_County_Crime.sqlite')

# push dataframes to tables in the database
county_output.to_sql('county_data', conn, if_exists='replace', index=False)
city_output.to_sql('city_data', conn, if_exists='replace', index=False)
county_rates.to_sql('county_rates', conn, if_exists='replace', index=False)

In [39]:
# test query to county_data
pd.read_sql('select * from county_data', conn).head()

,County,County_Type,Lat,Lng,Total_Crime,Violent_Crime,Murder_And_Nonnegligent_Manslaughter,Forcible_Rape,Robbery,Aggravated_Assault,Property_Crime,Burglary,Larceny_Theft,Motor_Vehicle_Theft,Total_Officers,Population,Land_Area
0,Alameda,Metropolitan,37.601689,-121.719546,57855,10308,132,519,4767,4890,47547,11708,26822,9017,2751,1510271,739
1,Butte,Metropolitan,39.625395,-121.537000,6215,751,7,89,153,502,5464,1826,3265,373,260,220000,1636
2,Contra Costa,Metropolitan,37.853409,-121.901795,32748,4257,65,187,1525,2480,28491,7546,15494,5451,1612,1049025,715
3,El Dorado,Metropolitan,38.742638,-120.435763,3099,485,3,34,66,382,2614,1112,1460,42,230,181058,1707
4,Fresno,Metropolitan,36.985898,-119.232078,43346,4904,63,181,1358,3302,38442,8982,22680,6780,1793,930450,5957


In [40]:
# test query to city_data
pd.read_sql('select * from city_data', conn).head()

,City,Lat,Lng,County,Total_Crime,Violent_Crime,Murder_And_Nonnegligent_Manslaughter,Forcible_Rape,Robbery,Aggravated_Assault,Property_Crime,Burglary,Larceny_Theft,Motor_Vehicle_Theft,Total_Officers
0,Alameda,37.779872,-122.282185,Alameda,2137,173,1,15,74,83,1964,389,1360,215,85
1,Albany,37.886704,-122.297756,Alameda,619,37,0,5,24,8,582,95,392,95,25
2,Alhambra,34.095287,-118.127015,Los Angeles,2260,185,3,8,82,92,2075,345,1460,270,84
3,Alturas,41.487115,-120.542456,Modoc,68,5,0,2,0,3,63,28,30,5,7
4,Anaheim,33.836593,-117.914301,Orange,9634,1161,7,88,492,574,8473,1594,5869,1010,389


In [47]:
# test query to county_rates
pd.read_sql('select * from county_rates', conn).head()

,County,County_Type,Lat,Lng,Total_Crime,Violent_Crime,Murder_And_Nonnegligent_Manslaughter,Forcible_Rape,Robbery,Aggravated_Assault,Property_Crime,Burglary,Larceny_Theft,Motor_Vehicle_Theft,Total_Officers,Population,Land_Area
0,Alameda,Metropolitan,37.601689,-121.719546,383.077,68.253,0.874,3.436,31.564,32.378,314.824,77.523,177.597,59.705,2751,1510271,739
1,Butte,Metropolitan,39.625395,-121.537000,282.500,34.136,0.318,4.045,6.955,22.818,248.364,83.000,148.409,16.955,260,220000,1636
2,Contra Costa,Metropolitan,37.853409,-121.901795,312.176,40.581,0.620,1.783,14.537,23.641,271.595,71.933,147.699,51.963,1612,1049025,715
3,El Dorado,Metropolitan,38.742638,-120.435763,171.161,26.787,0.166,1.878,3.645,21.098,144.374,61.417,80.637,2.320,230,181058,1707
4,Fresno,Metropolitan,36.985898,-119.232078,465.861,52.706,0.677,1.945,14.595,35.488,413.155,96.534,243.753,72.868,1793,930450,5957
